In [56]:
import time
from __future__ import division
from web3 import Web3, HTTPProvider
import json

import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
seaborn.despine()
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

# Connect to the network
web3 = Web3(HTTPProvider('http://localhost:8545'))

# Get contract signature
with open('../ethereum/build/contracts/Market.json') as energy_file:    
    energy_json = json.load(energy_file)
contract_abi = energy_json['abi']
network_id = energy_json['networks'].keys()[-1]
contract_address = energy_json['networks'][network_id]['address']

# List accounts and set default account
accounts = web3.eth.accounts
web3.eth.defaultAccount = accounts[0]
contract = web3.eth.contract(contract_abi, contract_address)
energy_posted_event = contract.on('energy_posted_event', filter_params={'fromBlock': 'earliest'})
market_cleared_event = contract.on('market_cleared_event', filter_params={'fromBlock': 'earliest'})
bill_sent_event = contract.on('bill_sent_event', filter_params={'fromBlock': 'earliest'})

In [57]:
print('Number of accounts ' + str(len(accounts)))

# Add all participants
for account in accounts:
    trans_hash = contract.transact({'from': account, 'to': contract_address, 'gas': 900000}).add_participant()
    
# Check how many participants have been added
print('Number of participants ' + str(contract.call().number_of_participant()))

Number of accounts 10
Number of participants 10


In [58]:
# Inputs
energy_balances = [100, 500, 600, 200, 300, -100, -100, -200, -25, -50]
print('Total energy balance ' + str(sum(energy_balances)) + ' wh')
assert(len(accounts) == len(energy_balances))

# Post consumption and production for each account
for account, energy_balance in zip(accounts, energy_balances):
    temp = {'from': account, 'to': contract_address, 'gas': 900000}
    contract.transact(temp).post_energy_balance(int(energy_balance))

# Clear the market
contract.transact({'to': contract_address, 'gas': 900000}).clear_market()

# Print results

Total energy balance 1225 wh


u'0x1e045a221595b724da9898d0b400b66e7a29547e33c1de53f2c709c56acea618'

In [59]:
market_cleared_event.get(only_changes=False)

[{'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_buy': 114,
   u'_gen': 475,
   u'_prod': 1700,
   u'_sell': 150,
   u'ratio': 357},
  'blockHash': u'0x6b7933ebd71f20b0a22f4f9d4ed7364eadeb3c77fb7bd6afee79a5586bcff0e8',
  'blockNumber': 28,
  'event': u'market_cleared_event',
  'logIndex': 0,
  'transactionHash': u'0x1e045a221595b724da9898d0b400b66e7a29547e33c1de53f2c709c56acea618',
  'transactionIndex': 0}]

In [60]:
energy_posted_event.get(only_changes=False)

[{'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x0d7c9010b492a0a265e37396ec1546ccb568af26',
   u'_value': 100},
  'blockHash': u'0x3836ea20b45335098cdb05b76be0cfca5cd51c76867bac96c2de49b79e481250',
  'blockNumber': 18,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0x3fe9030ed1eba894ca4267c7a4f339c08825738ad48cd892f470cb85af283d30',
  'transactionIndex': 0},
 {'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x4326c3abc7eadd989b7b3e73a5928676cf586e68',
   u'_value': 500},
  'blockHash': u'0x0dac4ceb4b952c1183d8c6f6e7b3f9ebdea6c35b0a7999d84a9050ac8dcbf0d0',
  'blockNumber': 19,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0xb5654d14e659c5d7cbdd91ab2750ff680734409d3a4da4a6c3294f8617e00de1',
  'transactionIndex': 0},
 {'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x319575916210ea91a26f9863058fb40068249bb3',
   u'_value': 60

In [61]:
bill_sent_event.get(only_changes=False)

[{'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x0d7c9010b492a0a265e37396ec1546ccb568af26',
   u'_value': 11400},
  'blockHash': u'0x6b7933ebd71f20b0a22f4f9d4ed7364eadeb3c77fb7bd6afee79a5586bcff0e8',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 1,
  'transactionHash': u'0x1e045a221595b724da9898d0b400b66e7a29547e33c1de53f2c709c56acea618',
  'transactionIndex': 0},
 {'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x4326c3abc7eadd989b7b3e73a5928676cf586e68',
   u'_value': 57000},
  'blockHash': u'0x6b7933ebd71f20b0a22f4f9d4ed7364eadeb3c77fb7bd6afee79a5586bcff0e8',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 2,
  'transactionHash': u'0x1e045a221595b724da9898d0b400b66e7a29547e33c1de53f2c709c56acea618',
  'transactionIndex': 0},
 {'address': u'0x87259acc4d5c068c56c708e3fbd7a3bd2369a23c',
  'args': {u'_target': u'0x319575916210ea91a26f9863058fb40068249bb3',
   u'_value': 68400}